# Calculate velocity and latent time using TFvelo

Notebook benchmarks velocity and latent time inference using TFvelo on dyngen-generated data.

## Library imports

In [1]:
import pandas as pd
import os

from sklearn.metrics import roc_auc_score
import anndata as ad
import scvelo as scv
from cellrank.kernels import VelocityKernel

import sys

sys.path.insert(0, "/lustre/groups/ml01/workspace/yifan.chen/TFvelo/")
import TFvelo as TFv
import numpy as np
import torch
import scipy

from rgv_tools import DATA_DIR
from rgv_tools.benchmarking import get_time_correlation, get_velocity_correlation, set_output

## Function definitions

In [2]:
def run_TFvelo(adata):
    """TFvelo pipeline."""
    adata.X = adata.X.toarray()
    n_gene = adata.shape[1]
    adata.varm["TFs"] = np.full([n_gene, n_gene], "blank")
    adata.varm["TFs"] = adata.varm["TFs"].astype("U10")

    adata.varm["TFs_id"] = np.full([n_gene, n_gene], -1)
    adata.varm["TFs_times"] = np.full([n_gene, n_gene], 0)
    adata.varm["TFs_correlation"] = np.full([n_gene, n_gene], 0.0)
    adata.varm["knockTF_Log2FC"] = np.full([n_gene, n_gene], 0.0)
    adata.var["n_TFs"] = np.zeros(n_gene, dtype=int)

    gene_names = adata.var_names.tolist()  # all genes as targets
    all_TFs = list(adata.var_names[adata.var["is_tf"]])  # select TFs

    for TF_name in all_TFs:
        TF_idx = gene_names.index(TF_name)
        TF_expression = np.ravel(adata[:, TF_name].X)

        for target in gene_names:
            target_idx = gene_names.index(target)
            if target == TF_name:
                continue

            if TF_name in adata.varm["TFs"][target_idx]:
                ii = list(adata.varm["TFs"][target_idx]).index(TF_name)
                adata.varm["TFs_times"][target_idx, ii] += 1
                continue
            target_expression = np.ravel(adata[:, target].X)
            flag = (TF_expression > 0) & (target_expression > 0)  # consider all possible regulation
            if flag.sum() < 2:
                correlation = 0
            else:
                correlation, _ = scipy.stats.spearmanr(target_expression[flag], TF_expression[flag])

            tmp_n_TF = adata.var["n_TFs"][target_idx]
            adata.varm["TFs"][target_idx][tmp_n_TF] = TF_name
            adata.varm["TFs_id"][target_idx][tmp_n_TF] = TF_idx
            adata.varm["TFs_times"][target_idx, tmp_n_TF] = 1
            adata.varm["TFs_correlation"][target_idx, tmp_n_TF] = correlation
            adata.var["n_TFs"][target_idx] += 1

    TFv.tl.recover_dynamics(
        adata,
        n_jobs=64,
        max_iter=20,
        var_names="all",
        WX_method="lsq_linear",
        WX_thres=20,
        n_top_genes=adata.shape[1],
        fit_scaling=True,
        use_raw=0,
        init_weight_method="ones",
        n_time_points=1000,
    )
    n_cells = adata.shape[0]
    expanded_scaling_y = np.expand_dims(np.array(adata.var["fit_scaling_y"]), 0).repeat(n_cells, axis=0)
    adata.layers["velocity"] = adata.layers["velo_hat"] / expanded_scaling_y
    return adata

## Constants

In [3]:
DATASET = "dyngen"

In [4]:
COMPLEXITY = "complexity_1"

In [5]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / COMPLEXITY / "results").mkdir(parents=True, exist_ok=True)

In [6]:
SAVE_DATASETS = True
if SAVE_DATASETS:
    (DATA_DIR / DATASET / COMPLEXITY / "trained_tfvelo").mkdir(parents=True, exist_ok=True)

## Velocity pipeline

In [7]:
velocity_correlation_all = []
velocity_correlation = []
grn_correlation = []

cnt = 0
for filename in (DATA_DIR / DATASET / COMPLEXITY / "processed").iterdir():
    torch.cuda.empty_cache()
    if filename.suffix != ".zarr":
        continue

    simulation_id = int(filename.stem.removeprefix("simulation_"))
    print(f"Run {cnt}, dataset {simulation_id}.")

    adata = ad.io.read_zarr(filename)

    if "spliced" in adata.layers:
        adata.layers["total"] = adata.layers["spliced"].todense() + adata.layers["unspliced"].todense()
    elif "new" in adata.layers:
        adata.layers["total"] = np.array(adata.layers["total"].todense())
    else:
        adata.layers["total"] = adata.X
    adata.layers["total_raw"] = adata.layers["total"].copy()

    TFv.pp.moments(adata, n_pcs=None, n_neighbors=None)

    adata = run_TFvelo(adata)

    # save data
    adata.write_zarr(DATA_DIR / DATASET / COMPLEXITY / "trained_tfvelo" / f"trained_{simulation_id}.zarr")

    velo_corr = get_velocity_correlation(
        ground_truth=adata.layers["true_velocity"], estimated=adata.layers["velocity"], aggregation=None
    )

    velocity_correlation_all.append(velo_corr)

    velo_corr = np.array(velo_corr)

    velocity_correlation.append(np.mean(velo_corr[~np.isnan(velo_corr)]))

    grn_true = adata.uns["true_skeleton"]
    grn_sc_true = adata.uns["true_sc_grn"]

    grn_estimate = pd.DataFrame(0, index=adata.var_names, columns=adata.var_names)
    grn_estimate.loc[:, adata.var_names[adata.var["is_tf"]]] = adata.varm["fit_weights_final"]
    grn_estimate = np.array(grn_estimate)

    grn_auroc = []
    for cell_id in range(adata.n_obs):
        ground_truth = grn_sc_true[:, :, cell_id]

        if ground_truth.sum() > 0:
            ground_truth = ground_truth.T[np.array(grn_true.T) == 1]
            ground_truth[ground_truth != 0] = 1

            estimated = grn_estimate[np.array(grn_true.T) == 1]
            estimated = np.abs(estimated)

            number = min(10000, len(ground_truth))
            estimated, index = torch.topk(torch.tensor(estimated), number)

            grn_auroc.append(roc_auc_score(ground_truth[index], estimated))

    grn_correlation.append(np.mean(grn_auroc))

    cnt += 1

Run 0, dataset 29.
computing moments based on connectivities
    finished (0:00:00) --> added 
    'M_total', moments of total abundances (adata.layers)
recovering dynamics (using 64/128 cores)


  0%|          | 0/362 [00:00<?, ?gene/s]

Processing 0/6 Target17
0/6 Target17 FINISHED with n_TFs: 25
Processing 1/6 Target26
1/6 Target26 FINISHED with n_TFs: 25
Processing 2/6 Target32
2/6 Target32 FINISHED with n_TFs: 25
Processing 3/6 Target36
3/6 Target36 FINISHED with n_TFs: 25
Processing 4/6 Target43
4/6 Target43 FINISHED with n_TFs: 25
Processing 5/6 Target46
5/6 Target46 FINISHED with n_TFs: 25
Processing 0/6 C1_TF1
0/6 C1_TF1 FINISHED with n_TFs: 24
Processing 1/6 C1_TF2
1/6 C1_TF2 FINISHED with n_TFs: 24
Processing 2/6 C2_TF1
2/6 C2_TF1 FINISHED with n_TFs: 24
Processing 3/6 C3_TF2
3/6 C3_TF2 FINISHED with n_TFs: 24
Processing 4/6 C4_TF1
4/6 C4_TF1 FINISHED with n_TFs: 24
Processing 5/6 C4_TF2
5/6 C4_TF2 FINISHED with n_TFs: 24
Processing 0/6 A5_TF1
0/6 A5_TF1 FINISHED with n_TFs: 24
Processing 1/6 B1_TF1
1/6 B1_TF1 FINISHED with n_TFs: 24
Processing 2/6 B2_TF1
2/6 B2_TF1 FINISHED with n_TFs: 24
Processing 3/6 B2_TF2
3/6 B2_TF2 FINISHED with n_TFs: 24
Processing 4/6 B4_TF1
4/6 B4_TF1 FINISHED with n_TFs: 24
Process

  0%|          | 0/664 [00:00<?, ?gene/s]

Processing 0/6 Target356
0/6 Target356 FINISHED with n_TFs: 25
Processing 1/6 Target357
1/6 Target357 FINISHED with n_TFs: 25
Processing 2/6 Target359
2/6 Target359 FINISHED with n_TFs: 25
Processing 3/6 Target360
3/6 Target360 FINISHED with n_TFs: 25
Processing 4/6 Target361
4/6 Target361 FINISHED with n_TFs: 25
Processing 5/6 Target362
5/6 Target362 FINISHED with n_TFs: 25
Processing 0/6 Target302
0/6 Target302 FINISHED with n_TFs: 25
Processing 1/6 Target305
1/6 Target305 FINISHED with n_TFs: 25
Processing 2/6 Target306
2/6 Target306 FINISHED with n_TFs: 25
Processing 3/6 Target308
3/6 Target308 FINISHED with n_TFs: 25
Processing 4/6 Target310
4/6 Target310 FINISHED with n_TFs: 25
Processing 5/6 Target311
5/6 Target311 FINISHED with n_TFs: 25
Processing 0/6 Target168
0/6 Target168 FINISHED with n_TFs: 25
Processing 1/6 Target169
1/6 Target169 FINISHED with n_TFs: 25
Processing 2/6 Target170
2/6 Target170 FINISHED with n_TFs: 25
Processing 3/6 Target172
3/6 Target172 FINISHED with n_

...

Processing 0/9 Target651
0/9 Target651 FINISHED with n_TFs: 49
Processing 1/9 Target652
1/9 Target652 FINISHED with n_TFs: 49
Processing 2/9 Target653
2/9 Target653 FINISHED with n_TFs: 49
Processing 3/9 Target654
3/9 Target654 FINISHED with n_TFs: 49
Processing 4/9 Target655
4/9 Target655 FINISHED with n_TFs: 49
Processing 5/9 Target656
5/9 Target656 FINISHED with n_TFs: 49
Processing 6/9 Target657
6/9 Target657 FINISHED with n_TFs: 49
Processing 7/9 Target658
7/9 Target658 FINISHED with n_TFs: 49
Processing 8/9 Target659
8/9 Target659 FINISHED with n_TFs: 49
Processing 0/5 Target373
0/5 Target373 FINISHED with n_TFs: 29
Processing 1/5 Target380
1/5 Target380 FINISHED with n_TFs: 29
Processing 2/5 Target382
2/5 Target382 FINISHED with n_TFs: 29
Processing 3/5 Target384
3/5 Target384 FINISHED with n_TFs: 29
Processing 4/5 Target385
4/5 Target385 FINISHED with n_TFs: 29
Processing 0/8 Target67
0/8 Target67 FINISHED with n_TFs: 37
Processing 3/8 Target70
3/8 Target70 FINISHED with n_TFs:

  0%|          | 0/208 [00:00<?, ?gene/s]

Processing 0/4 Target702
0/4 Target702 FINISHED with n_TFs: 29
Processing 1/4 Target704
1/4 Target704 FINISHED with n_TFs: 29
Processing 2/4 Target705
2/4 Target705 FINISHED with n_TFs: 29
Processing 3/4 Target706
3/4 Target706 FINISHED with n_TFs: 29
Processing 0/8 Target411
0/8 Target411 FINISHED with n_TFs: 37
Processing 1/8 Target412
1/8 Target412 FINISHED with n_TFs: 37
Processing 2/8 Target419
2/8 Target419 FINISHED with n_TFs: 37
Processing 3/8 Target423
3/8 Target423 FINISHED with n_TFs: 37
Processing 4/8 Target424
4/8 Target424 FINISHED with n_TFs: 37
Processing 5/8 Target425
5/8 Target425 FINISHED with n_TFs: 37
Processing 6/8 Target426
6/8 Target426 FINISHED with n_TFs: 37
Processing 7/8 Target427
7/8 Target427 FINISHED with n_TFs: 37
Processing 0/4 Target782
0/4 Target782 FINISHED with n_TFs: 29
Processing 1/4 Target786
1/4 Target786 FINISHED with n_TFs: 29
Processing 2/4 Target787
2/4 Target787 FINISHED with n_TFs: 29
Processing 3/4 Target788
3/4 Target788 FINISHED with n_

  0%|          | 0/494 [00:00<?, ?gene/s]

Processing 0/6 Target789
0/6 Target789 FINISHED with n_TFs: 41
Processing 1/6 Target790
1/6 Target790 FINISHED with n_TFs: 41
Processing 3/6 Target797
3/6 Target797 FINISHED with n_TFs: 41
Processing 0/9 Target49
0/9 Target49 FINISHED with n_TFs: 49
Processing 3/9 Target52
3/9 Target52 FINISHED with n_TFs: 49
Processing 4/9 Target53
4/9 Target53 FINISHED with n_TFs: 49
Processing 6/9 Target56
6/9 Target56 FINISHED with n_TFs: 49
Processing 7/9 Target60
7/9 Target60 FINISHED with n_TFs: 49
Processing 8/9 Target67
8/9 Target67 FINISHED with n_TFs: 49
Processing 0/9 Target790
0/9 Target790 FINISHED with n_TFs: 49
Processing 1/9 Target791
1/9 Target791 FINISHED with n_TFs: 49
Processing 2/9 Target792
2/9 Target792 FINISHED with n_TFs: 49
Processing 3/9 Target795
3/9 Target795 FINISHED with n_TFs: 49
Processing 5/9 Target797
5/9 Target797 FINISHED with n_TFs: 49
Processing 6/9 Target798
6/9 Target798 FINISHED with n_TFs: 49
Processing 7/9 Target799
7/9 Target799 FINISHED with n_TFs: 49
Proc

In [8]:
pd.DataFrame({"velocity": velocity_correlation, "grn": grn_correlation})

,velocity,grn
0,0.046940,0.451943
1,0.027392,0.581518
2,0.011939,0.690335
3,-0.001472,0.650858
4,0.012958,0.651840
5,-0.002415,0.523596
6,-0.004171,0.652698
7,0.037092,0.504753
8,-0.010419,0.657961
9,0.024667,0.547364


In [9]:
if SAVE_DATA:
    pd.DataFrame({"velocity": velocity_correlation, "grn": grn_correlation}).to_parquet(
        path=DATA_DIR / DATASET / COMPLEXITY / "results" / "tfvelo_correlation.parquet"
    )

In [11]:
if SAVE_DATA:
    df = pd.DataFrame(velocity_correlation_all).T
    df.columns = df.columns.astype(str)
    df.to_parquet(path=DATA_DIR / DATASET / COMPLEXITY / "results" / "tfvelo_correlation_all.parquet")